In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import sklearn as skl
from sklearn.metrics import average_precision_score
import shap
import pickle

In [2]:

" -------------------------------------- Load Data ------------------------------"

path= os.path.join('..', '..', 'Data', 'Residuals_0.975_Transfer_scRNA_Synthetic_Cell_Proportion_10_Fraction.csv')
Synthetic_Dataset_Residual = pd.read_csv(path)
print(Synthetic_Dataset_Residual)


               Gene_ID  expression  Fraction  preferential_expression  \
0      ENSG00000205090    0.727463  0.000382                 0.727463   
1      ENSG00000134644  491.570708  0.166582                 1.403998   
2      ENSG00000173406  111.664480  0.038844                 0.184405   
3      ENSG00000171385    1.435765  0.000637                -0.269458   
4      ENSG00000163382   20.126039  0.007132                 0.203013   
...                ...         ...       ...                      ...   
75715  ENSG00000173653    0.000000  0.000000                -0.772685   
75716  ENSG00000206487    0.000000  0.000000                 0.000000   
75717  ENSG00000168658    0.000000  0.000000                -0.536038   
75718  ENSG00000069812    0.000000  0.000000                 0.000000   
75719  ENSG00000171403    0.000000  0.000000                 0.000000   

       tipa_min  tipa_mean  tipa_max  num_interactors_dif_mean  \
0           NaN        NaN       NaN                  0.0

In [3]:
"--------------------------------- Rename Columns  ---------------------------"

rename_dict = {'expression': 'Expression', 'Fraction':'Fraction',
               'tipa_max':'Biological processes', 'paralogs_ratio_highest_identity':'Paralogs compensation', 
               'preferential_expression':'Preferential expression', 'diff_net_max': 'Differential interactors', 
               'num_interactors':'PPIs'}

Synthetic_Dataset_Residual.rename(columns=rename_dict, inplace=True)


In [4]:
"--------------------------------- Process Data ---------------------------"

relevant_cols = [c for c in rename_dict.values() if c not in ['Gene_ID', 'Affected_Cell', 'Max_Standardized_Residuals', 'Cell_Type']]
print('relevant_cols', len(relevant_cols), relevant_cols)

Synthetic_Dataset_Residual[relevant_cols] = Synthetic_Dataset_Residual[relevant_cols].apply(pd.to_numeric, errors='coerce')
Synthetic_Dataset_Residual[relevant_cols] = Synthetic_Dataset_Residual[relevant_cols].fillna(Synthetic_Dataset_Residual[relevant_cols].median())



relevant_cols 7 ['Expression', 'Fraction', 'Biological processes', 'Paralogs compensation', 'Preferential expression', 'Differential interactors', 'PPIs']


In [5]:
"--------------------------------- Cell Groups ---------------------------"

cell_types = Synthetic_Dataset_Residual['Cell_Type'][Synthetic_Dataset_Residual['Affected_Cell'] == True].unique()
print('cell_types', len(cell_types), cell_types)
general_cells = set([c.split('-')[1].strip().replace('cells', '') for c in cell_types])
general_cells = [c for c in general_cells if 'neurons' not in c]
general_cells.append('neurons')
print('general_cells', len(general_cells), general_cells)



cell_types 70 ['Thymus-Thymocytes' 'Adrenal-Adrenocortical cells'
 'Adrenal-Lymphoid cells' 'Adrenal-Myeloid cells' 'Adrenal-Schwann cells'
 'Cerebellum-Astrocytes' 'Cerebellum-Granule neurons'
 'Cerebellum-Microglia' 'Cerebellum-Oligodendrocytes'
 'Cerebellum-Purkinje neurons' 'Cerebrum-Astrocytes'
 'Cerebrum-Excitatory neurons' 'Cerebrum-Inhibitory neurons'
 'Cerebrum-Limbic system neurons' 'Cerebrum-Microglia'
 'Cerebrum-Oligodendrocytes' 'Cerebrum-Vascular endothelial cells'
 'Eye-Amacrine cells' 'Eye-Astrocytes' 'Eye-Bipolar cells'
 'Eye-Corneal and conjunctival epithelial cells' 'Eye-Ganglion cells'
 'Eye-Horizontal cells' 'Eye-Lens fibre cells' 'Eye-Microglia'
 'Eye-Photoreceptor cells' 'Eye-Retinal pigment cells'
 'Eye-Skeletal muscle cells' 'Eye-Smooth muscle cells' 'Eye-Stromal cells'
 'Eye-Vascular endothelial cells' 'Heart-Cardiomyocytes'
 'Heart-Endocardial cells' 'Heart-Lymphatic endothelial cells'
 'Heart-Lymphoid cells' 'Heart-Myeloid cells' 'Heart-Smooth muscle cells'


In [6]:
precentage = 9
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 12))
results_dict = {'Cell_Type':[], 'ROC_AUC':[], 'PR_AUC':[], 'Disease_Causing_Genes':[]}


for gc in general_cells:

    print('@', gc)
    Test_set = Synthetic_Dataset_Residual[Synthetic_Dataset_Residual['Cell_Type'].str.contains(gc)]
    Train_set = Synthetic_Dataset_Residual[~Synthetic_Dataset_Residual['Cell_Type'].str.contains(gc)]
    test_positive = len(Test_set[Test_set['Affected_Cell']==True])
    X_train = Train_set[relevant_cols]
    y_train = Train_set['Affected_Cell']
    X_test = Test_set[relevant_cols]
    y_test = Test_set['Affected_Cell']

    model = RandomForestClassifier(random_state=1234)  # select the model
    model.fit(X_train, y_train)  # train the model
    y_pred = model.predict(X_test)  # predict the test data
    predictions_proba = model.predict_proba(X_test)
    clr = classification_report(y_test, y_pred, output_dict=True)
    # print(clr)
    precision = clr['True']['precision']
    recall_1 = clr['True']['recall']
    f1_score = clr['True']['f1-score']
    print('precision:', precision, 'recall: ', recall_1, 'f1_score: ', f1_score)
    print(model.classes_)
    # plot_ROC(y_test, predictions_proba[:, 1], model)
    roc_auc1 = roc_auc_score(y_test, predictions_proba[:, 1])
    pred_true = predictions_proba[:, 1]
    print('AUC = %0.2f' % roc_auc1)
    print('\n')
    fpr, tpr, _ = roc_curve(y_test, predictions_proba[:, 1], pos_label=model.classes_[1])
    prec, recall, _ = precision_recall_curve(y_test, predictions_proba[:, 1], pos_label=model.classes_[1])
#     pr_auc1 = average_precision_score(y_test, pred_true)
    pr_auc1 = auc(recall, prec)

    ax1.plot(fpr, tpr, label='%s ROC_AUC (area = %0.2f), Genes:%s' % (gc, roc_auc1, test_positive))
    ax2.plot(recall, prec, label='%s PR_AUC = %0.2f, Genes:%s' % (gc, pr_auc1, test_positive))

    results_dict['Cell_Type'].append(gc)
    results_dict['ROC_AUC'].append(roc_auc1)
    results_dict['PR_AUC'].append(pr_auc1)
    results_dict['Disease_Causing_Genes'].append(test_positive)

#     break

print(results_dict)
Results = pd.DataFrame.from_dict(results_dict)
print(Results)
path = os.path.join('..', '..', 'Results', 'Multi_Cellular_Model_LOOCV_AUC.csv')
Results.to_csv(path, index=None)

ax1.plot([0, 1], [0, 1], 'r--')
ax1.set_xlabel('1-Specificity(False Positive Rate)')
ax1.set_ylabel('Sensitivity(True Positive Rate)')
ax1.set_title('Receiver Operating Characteristic')
ax1.legend(loc="lower right", fontsize='small')

ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title('Precision-Recall curve')
ax2.axhline(y=1 / (precentage + 1), color='red', linestyle='--',
            label=r'Causal genes frequency = %0.2f' % (1 / (precentage + 1)))

ax2.legend(loc="lower right", fontsize='small')
plt.suptitle('General scRNA Model, Leave One Cell Out Residuals')

path = os.path.join('..', '..', 'Results',
                    'Multi_Cellular_Model_LOOCV_AUC_Fig.pdf')
plt.savefig(path)
# plt.show()
plt.close()


@ Ciliated epithelial 
precision: 0.7 recall:  0.5714285714285714 f1_score:  0.6292134831460674
[False  True]
AUC = 0.89


@ Myeloid 
precision: 0.36024844720496896 recall:  0.1039426523297491 f1_score:  0.16133518776077888
[False  True]
AUC = 0.75


@ Amacrine 
precision: 0.4166666666666667 recall:  0.058823529411764705 f1_score:  0.10309278350515463
[False  True]
AUC = 0.82


@ Ganglion 
precision: 0.5128205128205128 recall:  0.09049773755656108 f1_score:  0.15384615384615385
[False  True]
AUC = 0.80


@ Intestinal epithelial 
precision: 0.75 recall:  0.1 f1_score:  0.17647058823529416
[False  True]
AUC = 0.70


@ Skeletal muscle 
precision: 0.5294117647058824 recall:  0.16097560975609757 f1_score:  0.24688279301745633
[False  True]
AUC = 0.75


@ Oligodendrocytes
precision: 0.6037735849056604 recall:  0.11307420494699646 f1_score:  0.19047619047619047
[False  True]
AUC = 0.80


@ Cardiomyocytes
precision: 0.7096774193548387 recall:  0.2682926829268293 f1_score:  0.3893805309734514
[

In [8]:

"----------------------------- Model Explanation With SHAP ---------------------------"

X = Synthetic_Dataset_Residual[relevant_cols]
y = Synthetic_Dataset_Residual['Affected_Cell']

model = RandomForestClassifier(random_state=1234)  # select the model
model.fit(X, y)  # train the model

explainer = shap.TreeExplainer(model)

shap_values = explainer.shap_values(X)

shap.summary_plot(shap_values[1], X, show=False, plot_type='bar')

plt.tight_layout()
path = os.path.join('..', '..', 'Results', 'Multi_Cellular_Model_SHAP_Summery_Plot_bar.pdf')
plt.savefig(path)
plt.close()
